# Projeto Final - Sistema de Controle Financeiro

Deverá ser desenvolvido um sistema para controle financeiro que receba as movimentações e as armazena em um arquivo csv ou json.

O sistema deverá ser capaz de realizar as seguintes operações:

- **Criar** novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor.

- Os tipos podem ser despesas, receita ou investimento:
  - No caso de receita, o valor deve ser tratado como numerico e armazenado normalmente.
  - no caso de despesas o valor deve ser recebido como positivo, mas armazenado como negativo
  - No caso de investimento, deve ter uma informação a mais de 'Montante', em que será calculado quanto o dinheiro rendeu desde o dia que foi investido. Para essa finalidade utilize a seguinte formula: $M = C * (1 + i)^t$ ([Saiba mais](https://matematicafinanceira.org/juros-compostos/)), considere tudo em dias.
- **Ler** registros: Deverá ser possível consultar os registros por data, tipo ou valor.
- **Atualizar** registros: No caso de atualização, pode-se atualizar o valor, o tipo e a data deverá ser a de atualização do registro.
- **Deletar**: Deverá ser possível deletar o registro (caso necessário, considere o indice do elemento como ID)
- Crie uma função que atualize os valores de rendimento sempre que chamada
- Crie uma função exportar_relatorio, que seja possível exportar um relatorio final em csv ou json .
- Crie pelo menos uma função de agrupamento, que seja capaz de mostrar o total de valor baseado em alguma informação (mes, tipo...)
- Crie valores separados para identificar a data (dia, mes, ano)

---

Atenção:
- Não utilize a biblioteca pandas para resolução desse exercício
- Deem um nome criativo para a aplicação de vocês

# Nome Criativo para a aplicação

O aplicativo utiliza dois arquivos como base de dados:

- investimentos.csv

- movimentacoes.csv



Importar modulos

In [1]:
from src.utils.funcoes import *

In [2]:
PATH = './database'

## Criar registro

A função recebe o tipo de investimento, valor, data, e, no caso de investomentos, a taxa.

ela grava os valores nos seus respectivos arquivos .csv

In [20]:
criar_registro_movimentacao(tipo='investimento', valor=100,
                            ano=2021, mes=1, dia=3, taxa=0.01)

('movimentacoes.csv', ValueError('Tipo de movimentação inválido.'))

In [21]:

# lista = "./database/financas_aleatorias_20_linhas.csv"
# movimentacoes = movimentacao(input_terminal=False, lista_movimentacoes=lista)
# # movimentacoes = atualizar_registro(movimentacoes, 4,53, "receita")
#

In [22]:
# agrupar_movimentacoes(movimentacoes, "mes")

In [23]:
# agrupar_movimentacoes(movimentacoes, "tipo")

In [24]:
# extrato = extrato_conta(movimentacoes)
# relatorio_json = exportar_relatorio(extrato, 'json')
# relatorio_json

In [6]:
# extrato = extrato_conta(movimentacoes)
# relatorio_csv = exportar_relatorio(extrato, 'csv')
# relatorio_csv

'ID, Data,Tipo,Valor,Montante\n17,2023-01-23 00:00:00,investimento,1978.0,1978.0\n9,2023-01-25 00:00:00,receita,1719.0,1981.9579779999995\n13,2023-01-27 00:00:00,investimento,8640.0,10625.923875913977\n7,2023-02-07 00:00:00,receita,3163.0,10743.3952211511\n4,2023-02-14 00:00:00,investimento,1602.0,12420.82497539387\n15,2023-03-21 00:00:00,despesa,-892.0,12863.026189099266\n8,2023-04-05 00:00:00,despesa,-3217.0,13057.328069959229\n10,2023-04-06 00:00:00,despesa,-9511.0,13070.385398029186\n12,2023-07-04 00:00:00,investimento,2337.0,16623.350116119647\n11,2023-07-12 00:00:00,despesa,-494.0,16756.803302924014\n5,2023-07-20 00:00:00,despesa,-9500.0,16891.327859394773\n0,2023-08-03 00:00:00,investimento,3531.0,20660.349725646905\n19,2023-08-15 00:00:00,investimento,4863.0,25772.64206095673\n14,2023-09-08 00:00:00,despesa,-5819.0,26398.35115841499\n16,2023-09-22 00:00:00,investimento,6842.0,33612.33996006567\n18,2023-10-06 00:00:00,investimento,4078.0,38163.98371104799\n1,2023-10-27 00:00:00,